# Identifying network metrics that can distinguish biased citation networks

Author: Yuanxi Fu, Zhonghe Wan

Date: 2021-7-5

## Introduction

This notebook records an attempt to realize the idea we proposed in the CRB proposal: https://docs.google.com/document/d/1yo9_zd4YlRLKQTQSSJn_dYCfl4YPGyYA2wb65DkCu4s/edit. In particular, we wanted to construct a program to simulate unbiased counter part for observed biased citation network and use statistical testing to identify network metrics that can distinguish biased networks. Also, with improved algorithm to calculate the expected citation counts, we can improve the work in the ISSI paper.

>  The overall goal is to choose a network metric η that can distinguish biased citation networks (e.g., Table 1) from simulated, unbiased networks. To do this, I will follow established procedures for assessing the significance of network characteristics (Kolaczyk & Csárdi, 2020):  gather existing biased networks; generate an ensemble of simulated networks that mimic unbiased citation behavior; choose an initial network metric η and compute the value of η on the reported biased citation network to get ηobs and a set of model-generated unbiased networks to get {ηmodel}. With the distribution from {ηmodel}, we will be able to test whether η is significantly different between biased and unbiased networks (i.e., ηobs very unlikely to exist given the distribution of {ηmodel}).

## Load packages

In [1]:
import pandas as pd
from igraph import * # install igraph in anaconda: https://anaconda.org/conda-forge/python-igraph
# import cairo # install cairo and pychairo in anaconda: https://anaconda.org/anaconda/cairo; https://anaconda.org/anaconda/pycairo
import numpy as np
# import numpy.random as random
# import math
# from matplotlib import pyplot as plt
# from scipy.stats import bernoulli
# from statistics import stdev
# import scipy.stats

## Load data

We will use the following dataset of a biased ciation network:

de Vries, Ymkje Anna, & Munafò, M. (2016). [Dataset] Citation bias and selective focus on positive findings in the literature on 5-HTTLPR, life stress, and depression. University of Bristol. Retrieved January 28, 2021, http://doi.org/10.5523/BRIS.Z7JCONXFBMDR1JJ3T0W4K1HWN

In [2]:
# load the adjacency matrix
matrix = pd.read_csv('HTTLPR.csv')
matrix = matrix.iloc[:, 6:]
matrix

,Mossner,Caspi,Eley,Grabe_a,Kaufman_a,Gillespie,Kendler,Surtees,Sjoberg,Nakatani,...,Comasco_a,Cicchetti_b,Jenness,Scheid_b,Quinn,Grabe_b,Petersen,Beaver,Brown,Wilhelm_b
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-,X,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-,X,-,-,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,-,X,-,X,-,-,X,-,X,-,...,-,-,-,-,-,NaN,NaN,NaN,NaN,NaN
69,-,X,X,-,-,-,X,X,X,-,...,-,-,-,-,-,-,NaN,NaN,NaN,NaN
70,-,X,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,NaN,NaN,NaN
71,-,X,X,-,-,-,-,-,X,-,...,-,-,-,-,-,-,-,-,NaN,NaN


In [3]:
# load the attribute list
# easy for this file because the first 6 columns are in fact attribute list
attr_list = pd.read_csv('HTTLPR.csv', usecols=[0, 1, 2, 3, 4, 5])
attr_list['PaperID'] = attr_list.index

## Create a graph object g from the citation network data

In [4]:
# initialize the graph object
g = Graph(directed=True)
g.add_vertices(73)

# add attributes to the graph: study name: name; study outcome: outcome; YearOnline: year
# study = attr_list['Study'].to_list()
# outcome = attr_list['Outcome'].to_list()
g.vs['name'] = attr_list['Study']
g.vs["label"] = g.vs["name"]
g.vs['outcome'] = attr_list['Outcome']
g.vs['Year'] = attr_list['YearOnline']
g.vs['color'] = 'light blue'

# add edges from the adjacency matrix
edge_list = []
for i in range(0, matrix.shape[0]):
    for j in range(0, matrix.shape[1]):
        if isinstance(matrix.iloc[i, j], str):
            s = matrix.iloc[i, j]
            if s.replace(" ", "") == "X":
                edge_list.append((i, j))
                g.add_edge(source=i, target=j)

In [5]:
# total number of edges
len(g.es)

488

In [10]:
citation = pd.DataFrame({'name': g.vs['name'], 'citation': g.degree(mode='in')})
citation

,name,citation
0,Mossner,5
1,Caspi,65
2,Eley,37
3,Grabe_a,21
4,Kaufman_a,27
...,...,...
68,Grabe_b,0
69,Petersen,0
70,Beaver,0
71,Brown,0


In [7]:
g.vs.name()

AttributeError: 'VertexSeq' object has no attribute 'name'

## Create a "potential graph"

Create a "potential graph" (g_full) for the 5-HTTLPR in which all "potential citation paths" (Urlings et al., 2021) are connected . In our definition, a "potential citation path" exists between two publication if one paper is published one year or more earlier than another paper.

In [7]:
# potential graph
year_gap = 1

g_full = Graph(directed=True)
g_full.add_vertices(73)
g_full.vs['name'] = attr_list['Study']
g_full.vs["label"] = g_full.vs["name"]
g_full.vs['outcome'] = attr_list['Outcome']
g_full.vs['Year'] = attr_list['YearOnline']

full_edge_list=[]

for i in g_full.vs.indices:
    for j in g_full.vs.indices:
        if i>j and (g.vs[i]['Year']-g.vs[j]['Year']>=year_gap):
            full_edge_list.append((i,j))
            g_full.add_edge(source=i,target=j)

In [8]:
# number of edges in g_full
len(g_full.es)

2348

## Compute p(y)

Each paper in this dataset faces a pool of papers to choose from to cite ("potential references"). Under our definition, such a pool consists of papers published one year or more earlier than the citing paper. In the real world, the authors may be biased (i.e., they select a non-representative sample from the pool to cite). In this simulation, we will try to remove such a bias. We will compute p(y), number of performed citations divided by the total number of potential citations between papers published in year y and their pool of potential references. Using a Bernoulli random number generator, we will distribute the chance evenly among all potential references (i.e., each paper will enjoy an equal chance of p(y) to be cited).

In [12]:
generations = pd.unique(g.vs['Year'])
generations

array([2001, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012],
      dtype=int64)

In [13]:
performed_citation = [len(g.es.select(_source_in=g.vs.select(Year=year))) for year in generations]
performed_citation

[0, 0, 4, 13, 38, 81, 52, 81, 85, 91, 43]

In [14]:
potential_citation = [len(g_full.es.select(_source_in=g.vs.select(Year=year))) for year in generations]
potential_citation

[0, 1, 8, 24, 80, 198, 232, 407, 336, 660, 402]

### The first generation (2001) has no performed citation and no potential citation. Replace p(y) with NaN

In [15]:
p_y = [i/j for i, j in zip(performed_citation[1:], potential_citation[1:])]
p_y = [None] + p_y
p_y

[None,
 0.0,
 0.5,
 0.5416666666666666,
 0.475,
 0.4090909090909091,
 0.22413793103448276,
 0.19901719901719903,
 0.25297619047619047,
 0.13787878787878788,
 0.10696517412935323]

**NOTE**: p(y) first increases and then decreases over the years.

## Use p(y) to simulate unbiased networks

In [35]:
prob = pd.DataFrame({'year': generations,
                    'performed_citation': performed_citation,
                    'potential_citation': potential_citation,
                    'p(y)': p_y})
prob

,year,performed_citation,potential_citation,p(y)
0,2001,0,0,NaN
1,2003,0,1,0.000000
2,2004,4,8,0.500000
3,2005,13,24,0.541667
4,2006,38,80,0.475000
5,2007,81,198,0.409091
6,2008,52,232,0.224138
7,2009,81,407,0.199017
8,2010,85,336,0.252976
9,2011,91,660,0.137879


In [36]:
# insert the 2002 in the table (2002, 0, 0, NaN)
prob = prob.append({'year': 2002, 'performed_citation': 0, 'potential_citation': 0, 'p(y)': None}, ignore_index=True)
prob = prob.sort_values(by='year',ignore_index=True)
prob = prob.reindex()
prob

,year,performed_citation,potential_citation,p(y)
0,2001.0,0.0,0.0,NaN
1,2002.0,0.0,0.0,NaN
2,2003.0,0.0,1.0,0.000000
3,2004.0,4.0,8.0,0.500000
4,2005.0,13.0,24.0,0.541667
5,2006.0,38.0,80.0,0.475000
6,2007.0,81.0,198.0,0.409091
7,2008.0,52.0,232.0,0.224138
8,2009.0,81.0,407.0,0.199017
9,2010.0,85.0,336.0,0.252976


In [37]:
pool = pd.DataFrame({'year': generations,'number of publications in each year':attr_list['YearOnline'].value_counts(sort=False),
                    'number of publications before the year':0})
pool['number of publications before the year']=pool['number of publications in each year'].cumsum()-pool['number of publications in each year']
pool.index = range(prob.shape[0])
pool

ValueError: Length mismatch: Expected axis has 11 elements, new values have 12 elements

In [ ]:
expected = prob['p(y)']*pool['number of publications in each year']
expected = expected.to_frame('expected number of publications')
expected.drop(0)

In [ ]:
expected_indegree = expected[::-1]['expected number of publications'].cumsum().to_list()
expected_indegree = expected_indegree[::-1]
expected_indegree = expected_indegree[1:]
expected_indegree

In [ ]:
a = []
for i in range(expected.shape[0]-1):
    a.extend([expected_indegree[i]]*int(pool.iloc[i,]['number of publications in each year']))

a.extend([0]*6)
a

In [ ]:
expected_citation_df = pd.DataFrame({'node':g_full.vs['name'],'degree':0})
expected_citation_df['degree'] = [round(elem,3) for elem in a]
expected_citation_df

## Improve the ISSI work with better "expected citation counts"

In [ ]:
degree_ratio = pd.DataFrame(columns=['paperID','degree_ratio'])
g_degree = pd.Series(g.degree(mode=IN))
last_generation = []

for idx, degree in enumerate(g_degree):
    if g_degree[idx]!=0:
        degree_ratio = degree_ratio.append({'paperID': idx, 'degree_ratio': round(g_degree[idx] / expected_citation_df.iloc[idx,1],3)},
                            ignore_index=True)
    else:
        last_generation.append(idx) # not suppose to have citations because of the constructs

In [ ]:
# max
degree_ratio['degree_ratio'].max()

In [ ]:
bins = [0,0.5,1.0,1.5,2.0,2.5,3.0,3.5]
plt.hist(degree_ratio['degree_ratio'], bins=bins,color='blue', edgecolor='blue', linewidth=1, hatch="//",
         histtype='bar', alpha=0.5)
plt.xlabel('Ratio between Real and Theoretical Citations',fontsize=14)
plt.ylabel('Count',fontsize=14)
plt.show()

In [ ]:
low_cutoff = 0.3
high_cutoff = 0.9
idx_low = degree_ratio[degree_ratio['degree_ratio'] < low_cutoff].index.tolist()
idx_low = [int(item) for item in idx_low]

idx_high = degree_ratio[degree_ratio['degree_ratio'] > high_cutoff].index.tolist()
idx_high = [int(item) for item in idx_high]

In [ ]:
# network visualization
for node in g.vs.indices:
    if node in idx_low:
        g.vs[node]['citation_ratio'] = 'Low'
    elif node in idx_high:
        g.vs[node]['citation_ratio'] = 'High'
    elif node in last_generation:
        g.vs[node]['citation_ratio'] = 'Last Generation'
    else:
        g.vs[node]['citation_ratio'] = 'Average'

color_dict = {"High": "green", "Low": "pink", "Average": "light blue","Last Generation":"white"}
g.vs["color"] = [color_dict[ratio] for ratio in g.vs["citation_ratio"]]
plot(g,bbox=(600,600),margin=50,layout="kk",vertex_label_size=10,edge_arrow_size=0.5,edge_arrow_width=1.5).show()

In [ ]:
# print out paper with low ratio between real and expected citation counts
for node in idx_low:
    print(g.vs[node]['name'],g.vs[node]['Year'],g.vs[node]['outcome'])

## relative evidence coupling strength

In [ ]:
# initialize the data storage
coupling_strength = pd.DataFrame(columns=['node_1','node_2','ratio'])

# looping through each pair in the net
for node_1_ix in g.vs.indices:
    for node_2_ix in g.vs.indices:
        if node_2_ix > node_1_ix:
            node_1 = g.vs[node_1_ix]
            node_2 = g.vs[node_2_ix]
            node_1_set = set(g.neighbors(node_1, mode=OUT))
            node_2_set = set(g.neighbors(node_2, mode=OUT))
            if (len(node_1_set) + len(node_2_set)) != 0:
                strength = len(node_1_set.intersection(node_2_set)) / (
                        len(node_1_set) + len(node_2_set) - len(
                    node_1_set.intersection(node_2_set)))  # full is 1, no overlap is zero
                coupling_strength = coupling_strength.append({'node_1': node_1_ix,
                                         'node_2': node_2_ix,
                                         'ratio': strength},ignore_index=True)


In [ ]:
coupling_strength.describe()

In [ ]:
len(coupling_strength.loc[coupling_strength['ratio']>0.9,'ratio'])

In [ ]:
len(coupling_strength.loc[coupling_strength['ratio']<0.1,'ratio'])

In [ ]:
bins = np.linspace(0,1,11)  # fixed number of bins
plt.xlabel('Relative Evidence Coupling Strength',fontsize=14)
plt.ylabel('Count',fontsize=14)
plt.ylim([0,900])
plt.xticks(ticks=bins)
plt.hist(coupling_strength['ratio'], bins=bins, color='darkorange', edgecolor='darkorange', linewidth=1, histtype='bar', alpha=0.5, hatch="//")

In [ ]:
threshold_p = 0.3

sufficient_overlap = coupling_strength[coupling_strength['ratio'] > threshold_p]
sufficient_overlap_nodes = list(set(sufficient_overlap['node_1']).union(set(sufficient_overlap['node_2'])))

unique_nodes = list(set(g.vs.indices).difference(set(sufficient_overlap_nodes)))

sufficient_overlap_nodes = g.vs.select(sufficient_overlap_nodes)
unique_nodes = g.vs.select(unique_nodes)

In [ ]:
# visualization [any updates need to be here]
sufficient_overlap_nodes['color'] = 'light blue'
unique_nodes['color'] = 'pink'
plot(g, margin=50,bbox=(600,600),layout='kk').show()

In [ ]:
# print nodes names
for node in unique_nodes:
    print(node['name'],node['Year'],node['outcome'])
